# TP2 Robotica Movel - Potential Field

## Alunos
- Bernardo Nogueira Borges 2020006396
- Daniele Cassia Silva Diniz 2020076874


### Importando Libs

In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

client = RemoteAPIClient()
sim = client.require('sim')
np.set_printoptions(precision=3,suppress=True)

### Obtendo Objetos de Interesse

In [4]:
# Objetos de interesse
objects = [
    ("robot","gray"),
]

## Criando Potential Field

In [5]:
# Configuracao do robo no referencial global q = [x,y,theta]
robot = sim.getObject("/robot")

def getRobotQ():
    robotM = sim.getObjectMatrix(robot)
    return [robotM[3],robotM[7],math.atan2(robotM[1],robotM[0])]

q = getRobotQ()
print(f"q = {q}")


q = [12.85033196659206, 11.498903104382785, -1.57062771152732]


## Movendo o robô

In [6]:
print("TODO")

TODO
